In [1]:
import urllib.request
from urllib.request import urlopen
import json
import datetime
import csv
import time

In [2]:
app_id = "EAACEdEose0cBAEuviQUFwKGZCWMEvZAfDLl0Q6xQwzbVY5jaZBhnTXkZCSSld6Upf54dLXxaQjZBdxNQaFtZB0ZB299kulAVQsObjOPqYZABnrEGZBedXnsxPm7Q34gaFtmZAYq1N8JgmlPtv9OCSkLEjuEj0heJUZACBFKEFbCLwDByGHclpA4nZC5ZCZCLtdxXBes78ZD"

access_token = app_id

In [3]:
page_id = 'seedlysg'

In [4]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.7"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = json.loads(response.read())
    
    print (json.dumps(data, indent=4, sort_keys=True))
    

testFacebookPageData(page_id, access_token)

{
    "id": "460639650790741",
    "name": "Seedly"
}


In [5]:
def request_until_succeed(url):
    req = urllib.request.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib.request.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print (e)
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()

In [6]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.7"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print (json.dumps(data, indent=4, sort_keys=True))
    

testFacebookPageFeedData(page_id, access_token)

{
    "data": [
        {
            "created_time": "2018-06-06T11:43:41+0000",
            "id": "247996545592127_702922650099512",
            "message": "Trust is like the air we breathe \u2014 when it's present, nobody really notices; when it's absent, everybody notices. Warren Buffett On 18 May 2018, Wall Street Journal(WSJ) published a video titled \u201cReporting on Theranos and Elizabeth Holmes\u201d. The summary of the book \u2018Bad Blood\u2019: Investigative reporter, John Carreyrou, burst the bubble of  Silicon Valley blood-testing startup, Theranos. [ 314 more words ]\nhttp://hive-up.com/2018/06/06/trust-and-honesty-are-a-rare-commodity/"
        },
        {
            "created_time": "2018-06-04T09:39:06+0000",
            "id": "247996545592127_701626793562431",
            "message": "Ever heard of this term? Join us on our community learning page and share your thoughts with us! https://www.facebook.com/groups/hiveup.community/"
        },
        {
            "cr

In [6]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print (json.dumps(test_status, indent=4, sort_keys=True))

{
    "comments": {
        "data": [
            {
                "created_time": "2018-06-22T03:06:17+0000",
                "id": "882940588560643_882981931889842",
                "message": "And remember.. always check the URL in your browser."
            }
        ],
        "paging": {
            "cursors": {
                "after": "WTI5dGJXVnVkRjlqZAFhKemIzSTZAPRGd5T1RneE9UTXhPRGc1T0RReU9qRTFNamsyTXpZAM056ZAz0ZD",
                "before": "WTI5dGJXVnVkRjlqZAFhKemIzSTZAPRGd5T1RneE9UTXhPRGc1T0RReU9qRTFNamsyTXpZAM056ZAz0ZD"
            }
        },
        "summary": {
            "can_comment": true,
            "order": "chronological",
            "total_count": 1
        }
    },
    "created_time": "2018-06-22T01:59:56+0000",
    "id": "460639650790741_882940588560643",
    "likes": {
        "data": [],
        "paging": {
            "cursors": {
                "after": "MTAyMDc4OTg1NzA3NzE0MDkZD",
                "before": "MTAyMDc4OTg1NzA3NzE0MDkZD"
            },


In [7]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print (processed_test_status)

('460639650790741_882940588560643', b'(Public service announcement) Please stay vigilant! In the news, there has been more brash and believable tactics by phishing scammers.\n\nEssentially what they plan to do is to get you to click on a link which directs you to a private site which resembles closely the bank\xe2\x80\x99s iBanking site where they will get you to submit your login and password. Common operating modes happen via SMS or Email.\n\nThings to take note of: Random number, suspicious looking links with wrong extensions, spelling and grammar mistakes.', b"Photos from Seedly's post", 'photo', 'https://www.facebook.com/seedlysg/photos/a.500789290109110.1073741829.460639650790741/882935508561151/?type=3', '2018-06-21 20:59:56', 7, 1, 8)


In [8]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'w') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print ("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print ("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print ("\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime))


scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping seedlysg Facebook Page: 2018-06-22 12:53:20.043195

1000 Statuses Processed: 2018-06-22 12:53:57.918554

Done!
1018 Statuses Processed in 0:00:40.562654
